# 分散的に表現された terms をまとめる
# string2string パッケージを使用

In [1]:
## run the following to install string2string package if neccessary
#!pip install -U string2string
## run the following if string2string needs it
#!pip install -U ipywidgets

In [2]:
import sys, os, re
import pandas as pd
import ast

In [3]:
## settings
verbose           = False
## term
term_classes      = [ 'spell', 'sound' ]
term_class        = term_classes[0]
n_for_ngram       = 4
ngram_is_skippy   = True
if ngram_is_skippy:
    term_type     = f"skippy{n_for_ngram}gram"
else:
    term_type     = f"{n_for_ngram}gram"
if term_class == 'sound':
    term_type = "sn_" + term_type
else:
    term_type = "sp_" + term_type
print(f"term_type: {term_type}")

## gap mark
gap_mark          = "…"
print(f"gap_mark: {gap_mark}")

## accent handling
suppress_accents   = True
if term_class == 'sound':
    if suppress_accents:
        accent_status = "-unaccented"
    else:
        accent_stratus = "-accented"
else:
    accent_status = ""
print(f"accent_status: {accent_status}")

term_type: sp_skippy4gram
gap_mark: …
accent_status: 


In [4]:
## target language
## a key must be part of a file name 
target_lang_dict = {    'en_US' : 'English (US)',
                        'en_UK' : 'English (UK)',
                        'en_N_only' : 'English noun (WN)',
                        'en_V_only' : 'English verb (WN)',
                        'en_A_only' : 'English adj (WN)',
                        'en_R_only' : 'English adv (WN)',
                        'ar'    : 'Arabic',
                        'de'    : 'German',
                        'de_N_only' : 'German Nouns',
                        'de_non_N_only' : 'German Non-nouns',
                        'eo'    : 'Esperanto',
                        'es_ES' : 'Spanish (Spain)',
                        'es_MX' : 'Spanish (Mexico)',
                        'fi'    : 'Finnish',
                        'fr_FR' : 'French (France)',
                        'fr_QC' : 'French (Quebec)',
                        'is'    : 'Icelandic',
                        'ir'    : 'Irish',
                        'nl'    : 'Dutch',
                        'ro'    : 'Romanian',
                        'sw'    : 'Swahili' }
## proper language selection
target_lang_keys = [    'en_US', # 0
                        'en_UK', # 1
                        'en_N_only', # 2
                        'en_V_only', # 3
                        'en_A_only', # 4
                        'en_R_only', # 5
                        'ar', # 6
                        'de', # 7
                        'de_N_only', # 8
                        'de_non_N_only', # 9
                        'eo', 'es_ES', 'es_MX',
                        'fi', 'fr_FR', 'fr_QC',
                        'is', 'nl', 'ro', 'sw',
                        'ir' # This lacks sound
                    ]
## check
target_lang_key  = target_lang_keys[4]
print(f"target_lang_key: {target_lang_key}")
print(f"target lang: {target_lang_dict[target_lang_key]} [{target_lang_key}]")
## target_attr [effective only for Irish]
target_class = ""
#target_class = None # This causes an unrediable error
if target_lang_key == "ir":
    target_classes = [ 'adjectives', 'nouns', 'verbs' ]
    target_class = f"-{target_classes[3]}"
print(f"target_class: {target_class}")
lang_dir_name = target_lang_dict[target_lang_key].split()[0]
print(f"lang_dir_name: {lang_dir_name}")


target_lang_key: en_A_only
target lang: English adj (WN) [en_A_only]
target_class: 
lang_dir_name: English


In [5]:
## read data from a file
import pandas as pd
import ast
##
max_n_topics = 45
##
use_sample = True
if use_sample:
    #data_file = f"results/terms-by-topics-raw/samples/hdp{max_n_topics}_topics_raw.csv"
    data_file = f"results/terms-by-topics-raw/English/English-pair-topics90-sp_5gram-sn_5gram-unaccented.csv"
else:
    data_file = f"results/terms-by-topics-raw/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-topics{max_n_topics}-{term_type}{accent_status}.csv"
print(data_file)

## read a file and evaluate its content
raw_df = pd.read_csv(data_file, header = None)
for col in raw_df:
    raw_df[col] = raw_df[col].apply(ast.literal_eval) # evaluation is crucial
##
raw_df

results/terms-by-topics-raw/English/English-pair-topics90-sp_5gram-sn_5gram-unaccented.csv


,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,"(#gal :: #ɡəɫ, 0.004248156812815527)","(gton# :: ɪŋtə, 0.004291690380586001)","(land :: #ɫænd, 0.004342214301990682)","(carb :: #kɑɹ, 0.004061346962031881)","(tock :: stɑk, 0.00441558156620048)","(ble# :: bəɫ#, 0.005563951476536451)","(#van :: #væn, 0.003759224453624997)","(line# :: aɪn#, 0.005076014379245686)","(eck# :: bɛk#, 0.00373008329555262)","(odge :: ɫɑdʒ, 0.004731948599928874)",...,"(ling# :: ɫɪŋ#, 0.0036955379675354233)","(ille# :: vɪɫ#, 0.004282078839442345)","(erin :: ɝɪŋ#, 0.005185234375393777)","(oman :: oʊmə, 0.0059112803042083)","(#mess :: #mɛs, 0.0040403060957354625)","(nks# :: ŋks#, 0.005103190217820056)","(life :: #ɫaɪf, 0.005256599738198006)","(ider :: aɪdɝ#, 0.004816085920841073)","(lings :: ɪŋz#, 0.004245911162598869)","(wick :: wɪk#, 0.004443914195441757)"
1,"(erman :: tɝmə, 0.0032761726425191108)","(#ligh :: #ɫaɪt, 0.003862912175634734)","(enin :: ənɪŋ#, 0.0034972856783785596)","(stry# :: tɹi#, 0.004052233440539947)","(kman :: kmən, 0.0040961140819052)","(ingt :: ɪŋtə, 0.003620751695214526)","(#over :: oʊvɝ, 0.0037423630819394087)","(amtra :: mtɹæ, 0.005051521402055748)","(#far :: #fɑɹ, 0.0036475522155502123)","(#mai :: #meɪ, 0.0044755413224138434)",...,"(ranc :: æns#, 0.0035013575593613486)","(tal# :: təɫ#, 0.004030772258205348)","(atter :: ætɝ#, 0.0037860304054055318)","(vand :: vænd, 0.004355830836093759)","(utch :: kətʃ, 0.003825879705124558)","(ding :: ɪdɪŋ#, 0.003972810242264332)","(less :: tɫəs, 0.004591930732290849)","(idge# :: ɪdʒ#, 0.004275297650450445)","(ight :: ɫaɪt, 0.00363721160378063)","(ello :: ɛɫoʊ, 0.003821268929772094)"
2,"(owski :: ski#, 0.0032534475311765615)","(owsk :: ski#, 0.0035130428791565874)","(erman :: ɝmən#, 0.003424881241823422)","(#gra :: #ɡɹæ, 0.003982225838663347)","(oman :: oʊmə, 0.0035972365595806502)","(erlin :: ɝɫɪn, 0.0031587023466216006)","(#mel :: #mɛɫ, 0.00372423802351194)","(nsky :: nski, 0.0039700117434851336)","(rrow# :: ɹoʊ#, 0.0032662952524186405)","(ence# :: əns#, 0.003912906271680249)",...,"(#fre :: #fɹɛ, 0.003068433830260731)","(#gin :: #ɡɪn, 0.0036286063484179776)","(#sto :: stɔɹ, 0.0037161256396394025)","(annon :: ænən, 0.0033279224225545422)","(illin :: ɪɫɪŋ, 0.0032463516785429217)","(#rep :: #ɹip, 0.0038265393476635944)","(art# :: ɪnhɑ, 0.004392788676127281)","(ord# :: fɝd#, 0.004108505379615207)","(#van :: vænd, 0.0036320409576697664)","(tson :: tsən#, 0.0037034354305959154)"
3,"(#non :: #nɑnf, 0.003218828760648162)","(ted# :: stɪd, 0.00348225088567199)","(ned# :: ənd#, 0.0032949913338566695)","(ling# :: əɫɪŋ, 0.003622674870936606)","(derma :: ɝmən, 0.0034899171942140146)","(#lig :: #ɫaɪ, 0.0030363359529131553)","(righ :: ɹaɪt#, 0.003275270860145393)","(derm :: mən#, 0.003594761103043637)","(flan :: #fɫæn, 0.0032178975555614405)","(#galo :: #ɡəɫ, 0.003307218641964862)",...,"(#mcn :: məkn, 0.0030357677589957755)","(#amt :: tɹæk, 0.0034246137229986596)","(#beac :: #bit, 0.0036816270992873403)","(are# :: wɛɹ#, 0.0030018324767827373)","(ster :: stɝz#, 0.003176975489670759)","(rd's :: ɑɹdz#, 0.00360101305294283)","(#par :: #pɛɹə, 0.004116585827694997)","(ship# :: ʃɪp#, 0.00386899489171738)","(der# :: ndɝ#, 0.003221218003074198)","(man# :: tmən, 0.00345563399676543)"
4,"(utche :: tʃɝ#, 0.003214399682457863)","(#ent :: #ɛnɝ, 0.003356694588500677)","(ngs# :: ɫɪŋz, 0.003275517980179494)","(ping# :: pɪŋ#, 0.003573835190723687)","(ide# :: aɪd#, 0.0034172266039369163)","(#lif :: ɫaɪf, 0.002940242966582451)","(#ligh :: #ɫaɪt, 0.0030794745176119993)","(ande :: #vænd, 0.00358300723372953)","(flan :: #fɫæ, 0.003159939586298707)","(#home :: #hoʊ, 0.0032044035703892183)",...,"(ley# :: sɫi#, 0.0030033358217822588)","(#whi :: #hwɪ, 0.0030609085424518965)","(ick# :: wɪk#, 0.003517768511614072)","(#vand :: #væn, 0.0030001421144081938)","(rant :: ɝænt, 0.0031700257963251906)","(on's# :: tənz, 0.0031821267440594056)","(ide# :: aɪd#, 0.00378133316494361)","(#lan :: ɫænd, 0.003827321658400536)","(ckma :: kmən, 0.0031

# 境界記号の処理

In [6]:
## supplement boudaries: this modification improves alignment quality
## But this modification most likely to do harm to paired terms
use_paired_terms = True
supplement_boundaries = True
boundary_mark = "#"
if supplement_boundaries and not use_paired_terms:
    for col in raw_df:
        enc = [ (f"{gap_mark}{term}", score) if term[-1] == boundary_mark else (f"{gap_mark}{term}{gap_mark}", score) for term, score in raw_df[col] ]
        raw_df[col] = enc
##
raw_df

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,"(#gal :: #ɡəɫ, 0.004248156812815527)","(gton# :: ɪŋtə, 0.004291690380586001)","(land :: #ɫænd, 0.004342214301990682)","(carb :: #kɑɹ, 0.004061346962031881)","(tock :: stɑk, 0.00441558156620048)","(ble# :: bəɫ#, 0.005563951476536451)","(#van :: #væn, 0.003759224453624997)","(line# :: aɪn#, 0.005076014379245686)","(eck# :: bɛk#, 0.00373008329555262)","(odge :: ɫɑdʒ, 0.004731948599928874)",...,"(ling# :: ɫɪŋ#, 0.0036955379675354233)","(ille# :: vɪɫ#, 0.004282078839442345)","(erin :: ɝɪŋ#, 0.005185234375393777)","(oman :: oʊmə, 0.0059112803042083)","(#mess :: #mɛs, 0.0040403060957354625)","(nks# :: ŋks#, 0.005103190217820056)","(life :: #ɫaɪf, 0.005256599738198006)","(ider :: aɪdɝ#, 0.004816085920841073)","(lings :: ɪŋz#, 0.004245911162598869)","(wick :: wɪk#, 0.004443914195441757)"
1,"(erman :: tɝmə, 0.0032761726425191108)","(#ligh :: #ɫaɪt, 0.003862912175634734)","(enin :: ənɪŋ#, 0.0034972856783785596)","(stry# :: tɹi#, 0.004052233440539947)","(kman :: kmən, 0.0040961140819052)","(ingt :: ɪŋtə, 0.003620751695214526)","(#over :: oʊvɝ, 0.0037423630819394087)","(amtra :: mtɹæ, 0.005051521402055748)","(#far :: #fɑɹ, 0.0036475522155502123)","(#mai :: #meɪ, 0.0044755413224138434)",...,"(ranc :: æns#, 0.0035013575593613486)","(tal# :: təɫ#, 0.004030772258205348)","(atter :: ætɝ#, 0.0037860304054055318)","(vand :: vænd, 0.004355830836093759)","(utch :: kətʃ, 0.003825879705124558)","(ding :: ɪdɪŋ#, 0.003972810242264332)","(less :: tɫəs, 0.004591930732290849)","(idge# :: ɪdʒ#, 0.004275297650450445)","(ight :: ɫaɪt, 0.00363721160378063)","(ello :: ɛɫoʊ, 0.003821268929772094)"
2,"(owski :: ski#, 0.0032534475311765615)","(owsk :: ski#, 0.0035130428791565874)","(erman :: ɝmən#, 0.003424881241823422)","(#gra :: #ɡɹæ, 0.003982225838663347)","(oman :: oʊmə, 0.0035972365595806502)","(erlin :: ɝɫɪn, 0.0031587023466216006)","(#mel :: #mɛɫ, 0.00372423802351194)","(nsky :: nski, 0.0039700117434851336)","(rrow# :: ɹoʊ#, 0.0032662952524186405)","(ence# :: əns#, 0.003912906271680249)",...,"(#fre :: #fɹɛ, 0.003068433830260731)","(#gin :: #ɡɪn, 0.0036286063484179776)","(#sto :: stɔɹ, 0.0037161256396394025)","(annon :: ænən, 0.0033279224225545422)","(illin :: ɪɫɪŋ, 0.0032463516785429217)","(#rep :: #ɹip, 0.0038265393476635944)","(art# :: ɪnhɑ, 0.004392788676127281)","(ord# :: fɝd#, 0.004108505379615207)","(#van :: vænd, 0.0036320409576697664)","(tson :: tsən#, 0.0037034354305959154)"
3,"(#non :: #nɑnf, 0.003218828760648162)","(ted# :: stɪd, 0.00348225088567199)","(ned# :: ənd#, 0.0032949913338566695)","(ling# :: əɫɪŋ, 0.003622674870936606)","(derma :: ɝmən, 0.0034899171942140146)","(#lig :: #ɫaɪ, 0.0030363359529131553)","(righ :: ɹaɪt#, 0.003275270860145393)","(derm :: mən#, 0.003594761103043637)","(flan :: #fɫæn, 0.0032178975555614405)","(#galo :: #ɡəɫ, 0.003307218641964862)",...,"(#mcn :: məkn, 0.0030357677589957755)","(#amt :: tɹæk, 0.0034246137229986596)","(#beac :: #bit, 0.0036816270992873403)","(are# :: wɛɹ#, 0.0030018324767827373)","(ster :: stɝz#, 0.003176975489670759)","(rd's :: ɑɹdz#, 0.00360101305294283)","(#par :: #pɛɹə, 0.004116585827694997)","(ship# :: ʃɪp#, 0.00386899489171738)","(der# :: ndɝ#, 0.003221218003074198)","(man# :: tmən, 0.00345563399676543)"
4,"(utche :: tʃɝ#, 0.003214399682457863)","(#ent :: #ɛnɝ, 0.003356694588500677)","(ngs# :: ɫɪŋz, 0.003275517980179494)","(ping# :: pɪŋ#, 0.003573835190723687)","(ide# :: aɪd#, 0.0034172266039369163)","(#lif :: ɫaɪf, 0.002940242966582451)","(#ligh :: #ɫaɪt, 0.0030794745176119993)","(ande :: #vænd, 0.00358300723372953)","(flan :: #fɫæ, 0.003159939586298707)","(#home :: #hoʊ, 0.0032044035703892183)",...,"(ley# :: sɫi#, 0.0030033358217822588)","(#whi :: #hwɪ, 0.0030609085424518965)","(ick# :: wɪk#, 0.003517768511614072)","(#vand :: #væn, 0.0030001421144081938)","(rant :: ɝænt, 0.0031700257963251906)","(on's# :: tənz, 0.0031821267440594056)","(ide# :: aɪd#, 0.00378133316494361)","(#lan :: ɫænd, 0.003827321658400536)","(ckma :: kmən, 0.0031

# 解析

In [7]:
## integrate redundant terms by superposition
import term_handler

##
topics_df = raw_df

## run the following if you need to make effective your updates in the module
reload_module = False
if reload_module:
    import importlib
    importlib.reload(term_handler)

## Good paramterization is crucial for convergence. Here are two samples of good one for
## global and another for local alignments. NB: different values would generate unexitable loop.
## Acceptable setting for global alignment
global_align_params = {
            'gap_weight'      : -2, # This may not be smaller
            'match_weight'    :  3,
            'mismatch_weight' : -2,
            'theta'           :  0 # If it takes too long to complete, try out a larger value.
            }
## Acceptable setting for local alignment: Local alignment is more robust for variation
local_align_params = {
            'gap_weight'      : -2.5,
            'match_weight'    :  1,
            'mismatch_weight' : -2.5,
            'theta'           :  0 # If it takes too long to complete, try out a larger value.
            }

## settings for execution
align_globally = False
if align_globally:
    method = "globally"
else:
    method = "locally"
expansive      = True
greedy         = True # This attribute is largely incompatible with global alignment
detailed       = True
check          = False

## main
print(f"method: {method}")
print(f"greedy: {greedy}")
print(f"expansive: {expansive}")
print(f"detailed: {detailed}")
print(f"check: {check}")    
for tid in topics_df:
    topic_data = topics_df[tid]
    if check:
        print(topic_data)
    #
    terms_as_dict = { }
    for data in topic_data:
        term, score = data[0], data[1]
        try:
            val = float(score)
            terms_as_dict[str(term)] = val
        except ValueError:
            print(f"#{score}")
    if check:
        print(terms_as_dict)
    ## resurively update data
    T = list(terms_as_dict.keys())
    if align_globally:
        O = term_handler.derive_terms_by_superposition(T, align_globally = True, expansive = expansive, greedy = greedy, check = check, **global_align_params)
    else:
        O = term_handler.derive_terms_by_superposition(T, align_globally = False, expansive = expansive, greedy = greedy, check = check, **local_align_params)
    ## show result
    if detailed:
        T = sorted(T, key = lambda x: (len(x) - x.count(gap_mark) - x.count("[") - x.count("]")), reverse = True)
        print(f"topic {tid:03} has {len(T):03} raw terms: {T}")
    #
    O = [ x for x in O if not x in T ] # Filter out terms already in originals 
    O = sorted(O, key = lambda x: (len(x) - x.count(gap_mark) - x.count("[") - x.count("]")), reverse = True)
    print(f"topic {tid:03} found {len(O):03} {method} superposed terms: {O}")
##
print(f"# Topic-wise term integration for HDP/LDA {max_n_topics} topics via {target_lang_dict[target_lang_key]} {target_class}{term_type}{accent_status}") 

method: locally
greedy: True
expansive: True
detailed: True
check: False
topic 000 has 120 raw terms: ['nski# :: nski#', '#life :: #ɫaɪf', 'tman# :: tmən#', 'lland :: ɫənd#', 'nnon# :: ænən#', 'sters :: stɝz#', 'amtra :: #æmtɹ', "ton's :: ɹtənz", 'amtra :: æmtɹæ', 'erman :: tɝmə', 'owski :: ski#', '#non :: #nɑnf', 'utche :: tʃɝ#', 'nnick :: nɪk#', '#cas :: #kæsə', 'idge :: ɹɪdʒ#', 'den# :: ɫdən#', 'erso :: ɝsən#', 'ling :: əɫɪŋ#', 'nski# :: ɪnsk', 'llan :: ɫənd#', '#bol :: #boʊɫ', 'led# :: təɫd#', '#land :: #ɫæn', 'ers# :: ndɝz#', 'terma :: ɝmən', '#scar :: skɛɹ', '#chi :: #tʃɪt', 'elin# :: ɫɪn#', 'maste :: mæst', 'ancy# :: nsi#', 'hart :: nhɑɹt', 'line# :: ɫaɪn', 'wood# :: wʊd#', 'ship# :: ɝʃɪp', 'erma :: ɝmən#', 'ppert :: pɝt#', 'aro# :: ɑɹoʊ#', 'ters# :: stɝz', 'light :: ɫaɪt', 'gton# :: ŋtən', 'tman# :: tmən', 'ider :: aɪdɝ#', 'mela :: #mɛɫə', 'dge# :: ɹɪdʒ#', 'berg# :: bɝɡ#', '#swin :: #swɪ', 'rship :: ɝʃɪp', 'amtr :: #æmtɹ', 'nski :: ɪnski', '#rock :: #ɹɑk', 'insk :: ɪnski', '#su